# **Initial Exploration**

## Objectives

* Write here your notebook objective, for example, "Fetch data from Kaggle and save as raw data", or "engineer features for modelling"

## Inputs

* Write here which data or information you need to run the notebook 

## Outputs

* Write here which files, code or artefacts you generate by the end of the notebook 

## Additional Comments

* In case you have any additional comments that don't fit in the previous bullets, please state them here. 


---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'/workspace/parkrun-explore/jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'/workspace/parkrun-explore'

# Section 1

Section 1 content

In [4]:
import pandas as pd
df = pd.read_csv(f"inputs/datasets/raw/airruns.csv",index_col=0)
df.head()

,name,time,parkrun,race_num,date,year,is_male,med_age,location,co,no,no2,o3,so2,pm2_5,pm10,nh3,county,lat,lon
0,James BAKER,17.183333,hogmoorinclosure,180,2022-12-31,2022,1,47.0,Whitehill,193.6,0.0,4.16,67.23,2.0,3.56,4.59,0.06,Hampshire,51.097719,-0.864067
1,Jon FAIRS,19.483333,hogmoorinclosure,180,2022-12-31,2022,1,47.0,Whitehill,193.6,0.0,4.16,67.23,2.0,3.56,4.59,0.06,Hampshire,51.097719,-0.864067
2,Aaron HICKEY,20.750000,hogmoorinclosure,180,2022-12-31,2022,1,18.5,Whitehill,193.6,0.0,4.16,67.23,2.0,3.56,4.59,0.06,Hampshire,51.097719,-0.864067
3,Oliver BARLOW,21.233333,hogmoorinclosure,180,2022-12-31,2022,1,37.0,Whitehill,193.6,0.0,4.16,67.23,2.0,3.56,4.59,0.06,Hampshire,51.097719,-0.864067
4,Catherine SEAGER,21.600000,hogmoorinclosure,180,2022-12-31,2022,0,42.0,Whitehill,193.6,0.0,4.16,67.23,2.0,3.56,4.59,0.06,Hampshire,51.097719,-0.864067


In [ ]:
df.info()

In [5]:
# Drop atmospheric data, name, location and county, race no
df = pd.read_csv(f"inputs/datasets/raw/airruns.csv",index_col=0).drop(['co','no','no2','o3','so2','pm2_5','pm10','nh3', 'name','race_num','location', 'county'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4351832 entries, 0 to 4351831
Data columns (total 8 columns):
 #   Column   Dtype  
---  ------   -----  
 0   time     float64
 1   parkrun  object 
 2   date     object 
 3   year     int64  
 4   is_male  int64  
 5   med_age  float64
 6   lat      float64
 7   lon      float64
dtypes: float64(4), int64(2), object(2)
memory usage: 298.8+ MB


In [ ]:
from ydata_profiling import ProfileReport
pandas_report = ProfileReport(df=df, title='Cheeky Explore', minimal=True)
pandas_report.to_notebook_iframe()

___


# Target Encoding / Mean Encoding

In [8]:
from feature_engine.encoding import OneHotEncoder
encoder = OneHotEncoder(variables=['parkrun'], drop_last=False)
df_ohe = encoder.fit_transform(df)
print(df_ohe.shape)
df_ohe.head(3)

/workspace/.pyenv_mirror/user/current/lib/python3.11/site-packages/feature_engine/encoding/one_hot.py:277: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X[f"{feature}_{category}"] = np.where(X[feature] == category, 1, 0)
/workspace/.pyenv_mirror/user/current/lib/python3.11/site-packages/feature_engine/encoding/one_hot.py:277: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X[f"{feature}_{category}"] = np.where(X[feature] == category, 1, 0)
/workspace/.pyenv_mirror/user/current/lib/python3.11/site-packages/feature_engine/encoding/o

(4351832, 628)


,time,date,year,is_male,med_age,lat,lon,parkrun_hogmoorinclosure,parkrun_goole,parkrun_standrews,...,parkrun_pegwellbay,parkrun_oxford,parkrun_seafordbeach,parkrun_homewood,parkrun_gangerfarm,parkrun_californiacountry,parkrun_eastbourne,parkrun_greatsalterns,parkrun_frimleylodge,parkrun_dartford
0,17.183333,2022-12-31,2022,1,47.0,51.097719,-0.864067,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,19.483333,2022-12-31,2022,1,47.0,51.097719,-0.864067,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,20.750000,2022-12-31,2022,1,18.5,51.097719,-0.864067,1,0,0,...,0,0,0,0,0,0,0,0,0,0


___


## Correlation Study

In [10]:
corr_spearman = df_ohe.corr(method='spearman', numeric_only=False)['time'].sort_values(key=abs, ascending=False)[1:].head(10)
corr_spearman

ValueError: could not convert string to float: '2022-12-31'

# Push files to Repo

* In case you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
import os
try:
  # create here your folder
  # os.makedirs(name='')
except Exception as e:
  print(e)
